# Универсальные параметры запроса
Когда мы выгружали отчеты из Яндекс.Метрики, то каждый раз приходится разбираться со структурой JSON-ответа и писать циклы.

В этом шаге мы сделаем универсальную функцию, которая будет переводить таблицу в отчете API Метрики в набор листов. Для получения отчета вам нужно будет только задать нужный набор метрик, измерений и период выгрузки.

Также мы познакомимся с интересным методом передачи аргументов в функцию со странным названием kwargs.

Итак, сейчас наша задача сводится к следующей: по набору метрик и измерений получить отчет Яндекс.Метрики за заданный период в виде набора листов. Сейчас набор измерений и метрик не важен, выберем по три штуки (измерения: дата визита, тип устройства, браузер, метрики: визиты, посетители, просмотры страниц):

In [100]:
import requests
from yaml import load
from pprint import pprint
import pandas as pd
import numpy as np

In [3]:
dimensions = ['ym:s:date', 'ym:s:deviceCategory', 'ym:s:browser']
metrics = ['ym:s:visits', 'ym:s:users', 'ym:s:pageviews']

with open('./module12_files/yandex_config.yaml') as f:
    token = load(f)['access_token']
    
api_url = 'https://api-metrika.yandex.ru/stat/v1/data'

startDate, endDate = '2018-02-12', '2018-02-18'
counter = '21075004'

В веб-интерфейсе этот отчет выглядит так https://metrika.yandex.ru/stat/browsers?dimension_mode=list&period=2018-02-12%3A2018-02-18&id=21075004&stateHash=5a8ec43962c3c4110188cc2a
<img src='./module12_files/m8_b6_metrika_report.jpg'>

Начнем с функции, которая делает запрос к API.

**В текущей формулировке задачи у нас появляется проблема: мы заранее не знаем какие параметры будут переданы в запросе API.**

Например, сейчас в нашем запросе нет фильтра (параметр filters, который мы использовали в прошлом шаге), но он может понадобиться. Или может понадобиться изменить точность запроса (параметр accuracy). Также в функцию запроса могут передаваться и вспомогательные переменные, не касающиеся Метрики.

Для учета таких ситуаций используется следующий прием: в аргументах функции напишем **kwargs. И посмотрим что представляет из себя эта переменная:

In [14]:
def make_request(**kwargs):
    return requests.get(api_url, params=kwargs).json()

In [10]:
params = {
    'date1': startDate,
    'date2': endDate,
    'dimensions': ','.join(dimensions),
    'metrics': ','.join(metrics),
    'id': counter,
    'oauth_token': token,
    'accuracy': 1
}

In [18]:
data = make_request(**params)

# Автоматическое преобразование JSON в таблицу
Мы получили ответ в JSON-формате, который неплохо бы перевести в наглядную таблицу. Поскольку мы теперь заранее не знаем как называются столбцы и строки, то давайте сделаем универсальный алгоритм, который преобразует данные из JSON-формата в наглядный набор листов.

In [23]:
# dimensions

# Итоговая функция
Оформим наш цикл в функцию и уберем break:

In [39]:
def json_handler(data):
    """Преобразование ответа из JSON в список списков"""
    report = []
    for line in data['data']:
        line_metrics = line['metrics']
        line_dimensions = [value['name'] for value in line['dimensions']]
        report.append(line_dimensions + line_metrics)
        
    return report

In [50]:
# %%time

# pd.DataFrame(json_handler(data))

Wall time: 1 ms


,0,1,2,3,4,5
0,2018-02-17,Смартфоны,Chrome Mobile,138.0,109.0,260.0
1,2018-02-16,Смартфоны,Chrome Mobile,122.0,107.0,185.0
2,2018-02-17,Смартфоны,Mobile Safari,109.0,85.0,165.0
3,2018-02-18,Смартфоны,Chrome Mobile,107.0,80.0,224.0
4,2018-02-13,Смартфоны,Chrome Mobile,103.0,97.0,141.0
5,2018-02-14,Смартфоны,Chrome Mobile,95.0,87.0,135.0
6,2018-02-12,Смартфоны,Chrome Mobile,87.0,78.0,118.0
7,2018-02-16,Смартфоны,Mobile Safari,84.0,74.0,127.0
8,2018-02-15,Смартфоны,Chrome Mobile,77.0,72.0,130.0
9,2018-02-12,ПК,Google Chrome,69.0,65.0,113.0


# Упражнение
(1 возможный балл)
Посчитайте долю блокировщиков рекламы по источникам трафика за 12-18 февраля 2018. Не учитывайте строчки, в которых наличие блокировщиков не определено. В качестве метрики возьмите визиты. Измерение для блокировщиков рекламы ym:s:hasAdBlocker, источник трафика - ym:s:TrafficSource.

Какой источник имеет самую большую долю визитов с использованием блокировщиков рекламы?

In [93]:
params = {
    'date1': startDate,
    'date2': endDate,
    'dimensions': ','.join(['ym:s:date', 'ym:s:hasAdBlocker', 'ym:s:trafficSource']),
    'metrics': ','.join(['ym:s:visits']),
    'id': counter,
    'oauth_token': token,
    'accuracy': 1
}

In [94]:
data1 = make_request(**params)

In [142]:
df = pd.DataFrame(json_handler(data1), 
                  columns=['date', 'adblock', 'traffic', 'visits'])

df['adblock'] = df['adblock'].map(lambda x: np.nan 
                                  if x=='Не определено' 
                                  else x)
df.dropna(inplace=True)

df_pivot = df.pivot_table(values='visits', index='traffic', columns='adblock', aggfunc='sum').reset_index()

df_pivot['%'] = np.divide(df_pivot['Блокировщик используется'], 
                          df_pivot['Блокировщик не используется'])

In [146]:
df_pivot['traffic'].iloc[df_pivot['%'].idxmax()]

'Внутренние переходы'